# Mapping properties to dictionary and the other way around

### Converting properties to a dictionary

In [ ]:
import os
import sys
sys.path.append('../causalgraph/store/')
sys.path.insert(1, os.path.join(sys.path[0], '..'))
import json
# causalgraph imports
from causalgraph import Graph
import causalgraph.utils.owlready2_utils as owl2utils

In [ ]:
# Specify new sqlite db for this example
# delete old SQL-DB if exists (restart of kernel may be necessary):
sql_file_name='example1.sqlite3'
if os.path.exists(sql_file_name):
    os.remove(sql_file_name)
    print(f"Deleted old db with name {sql_file_name}")

In [ ]:
# init graph
G = Graph(sql_db_filename=sql_file_name)
# add creators
creator_node = G.add.individual_of_type("Creator", name_for_individual="master_creator")
# add nodes
G.add.causal_node("node_1")
G.add.causal_node("node_2")
G.add.causal_node("node_3")
G.add.causal_node("node_4_c", hasCreator=[creator_node])
# add edges
G.add.causal_edge("node_1", "node_2", "edge_1")
G.add.causal_edge("node_2", "node_3", "edge_2", confidence=0.5, time_lag_s=3.5)
G.add.causal_edge("node_3", "node_4_c", "edge_3_c", confidence=0.2, time_lag_s=1.5, hasCreator=[creator_node])

In [ ]:
# Convert properties of the whole cg graph to dict
graph_dict = G.map.all_individuals_to_dict()
print(json.dumps(graph_dict, indent=4, sort_keys=True))

## Updating properties from a dictionary

In [ ]:
sql_file_name='example2.sqlite3'
if os.path.exists(sql_file_name):
    os.remove(sql_file_name)
    print(f"Deleted old db with name {sql_file_name}")

In [ ]:
# init Graph
G_n = Graph(sql_db_filename=sql_file_name)
# add creators
creator_1 = G_n.add.individual_of_type("Creator", name_for_individual="creator_1")
# add nodes
G_n.add.causal_node("node_1")
G_n.add.causal_node("node_2")
G_n.add.causal_node("node_3")
G_n.add.causal_node("node_4")
# add edges
G_n.add.causal_edge("node_1", "node_2", "edge_1_c", time_lag_s=1, confidence=0.5, hasCreator=[creator_1])
G_n.add.causal_edge("node_3", "node_4", "edge_2_c", time_lag_s=5, confidence=0.25)

In [ ]:
graph_dict_old = G_n.map.all_individuals_to_dict()
graph_dict_new = {
    "creator_1": {
        "created": [
            "node_1",       # new
            "edge_1_c"      
        ],
        "type": "Creator"
    },
    "edge_1_c": {
        "cause": [
            "node_2"        # new
        ],
        "confidence": 1,    # new
        "creator": [
            "creator_1"
        ],
        "effect": [
            "node_4"        # new
        ],
        "time_lag_s": 1,
        "type": "CausalEdge"
    },
    "edge_2_c": {
        "cause": [
            "node_1"        # new
        ],
        "confidence": 0.5,  # new
        "effect": [
            "node_3"        # new
        ],
        "time_lag_s": 10,   # new
        "type": "CausalEdge"
    },
    "node_1": {
        "causing": [
            "edge_2_c"      # new
        ],
        "creator": [
            "creator_1"     # new
        ],                  
        "type": "CausalNode"
    },
    "node_2": {
        "causing": [
            "edge_1_c"      # new
        ],
        "type": "CausalNode"
    },
    "node_3": {
        "affected_by": [
            "edge_2_c"      # new
        ],
        "type": "CausalNode"
    },
    "node_4": {
        "affected_by": [
            "edge_1_c"      # new
        ],
        "type": "CausalNode"
    }
}


In [ ]:
G_n.draw.nx()